In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

df = pd.read_csv('/Users/erica/Desktop/Y3S2/DSE3101/Local/reslae_price_normalized_for_ML.csv')

df = df.drop(columns=['flat_model', 'building_age_2025', 'total_unemployment_rate', 
                      'Chinese', 'Malays', 'Indians', 'Others', 'fx_rate', 'floor_area_sqm'])
df_normalized_clean = df.copy()
def normalize(col):
    return (col - col.min()) / (col.max() - col.min()) if col.max() != col.min() else col

df_normalized_clean['month'] = pd.to_datetime(df_normalized_clean['month'])
df_normalized_clean['year'] = df_normalized_clean['month'].dt.year.astype(float)
df_normalized_clean['month_num'] = df_normalized_clean['month'].dt.month.astype(float)
df_normalized_clean = df_normalized_clean.drop(columns=['month'])
df_normalized_clean = df_normalized_clean.drop(columns = ['CPI (base 2024-12)'])

columns_to_normalize = ['inflation_rate (x100)', 'interest_rate', 'priv_prop',
                        'resident_unemployment_rate', 'month_num', 'year']

df_normalized_clean[columns_to_normalize] = df_normalized_clean[columns_to_normalize].apply(normalize)

categorical_features = ['town']
numerical_features = [
    'storey_range', 'remaining_lease',
    'lat', 'lon', 'nearest_mrt_distance', 'nearest_bus_distance',
    'education_score', 'shopping_score', 'food_score', 'recreation_score',
    'healthcare_score', 'inflation_rate (x100)',
    'resident_unemployment_rate',
    'interest_rate', 'avg_household_income', 'priv_prop', 'flat_type'
]
numerical_features.extend(['year', 'month_num'])
demographic_features = [
    'NoReligion', 'Buddhism', 'Taoism1', 'Islam', 'Hinduism', 'Sikhism',
    'Christianity_Catholic', 'Christianity_OtherChristians', 'OtherReligions'
]
numerical_features.extend(demographic_features)  # Add to numerical pipeline

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [9]:
df_sample = df_normalized_clean.sample(frac=0.2, random_state=42)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid to search
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [10, 20, 30],
    'regressor__min_samples_leaf': [1, 2, 4]
    }

grid_search = GridSearchCV(
    model,
    param_grid,
    cv=3,
    scoring={
        'MAE': 'neg_mean_absolute_error',
        'RMSE': 'neg_root_mean_squared_error',
        'R2': 'r2'
    },
    refit='MAE',  # Use MAE to choose the best model
    n_jobs=-1,
    verbose=2
)

# Split your data
X = df_sample.drop(columns=['resale_price'])
y = df_sample['resale_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best CV MAE:", -grid_search.cv_results_['mean_test_MAE'][grid_search.best_index_])
print("Best CV RMSE:", -grid_search.cv_results_['mean_test_RMSE'][grid_search.best_index_])
print("Best CV R2:", grid_search.cv_results_['mean_test_R2'][grid_search.best_index_])


Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__n_estimators=100; total time=  15.0s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__n_estimators=100; total time=  15.0s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__n_estimators=100; total time=  15.1s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=2, regressor__n_estimators=100; total time=  15.1s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=2, regressor__n_estimators=100; total time=  15.2s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=2, regressor__n_estimators=100; total time=  15.0s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__n_estimators=200; total time=  30.1s
[CV] END regressor__max_depth=10, regressor__min_samples_leaf=1, regressor__n_estimators=200; total time=  30.2s
[CV] END regressor__max_depth=10, r

In [11]:
best_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        max_depth=30,
        min_samples_leaf=1,
        n_estimators=200,
        random_state=42
    ))
])

In [12]:
import joblib
X_full = df_normalized_clean.drop(columns=['resale_price'])
y_full = df_normalized_clean['resale_price']
best_model.fit(X_full, y_full)
joblib.dump(best_model, 'best_resale_price_model.pkl')

['best_resale_price_model.pkl']